# SQL

Nous utiliserons le dialecte Transact-SQL (T-SQL) : [https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15](https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15)  

## Création BDD

In [ ]:
IF (EXISTS (SELECT [name] FROM [master].[sys].[databases] WHERE [name] = 'ImdbLike'))
    DROP DATABASE [ImdbLike]

CREATE DATABASE [ImdbLike]

GO

USE [ImdbLike]

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.089

## Schéma

Un SGBD =\> plusieurs BDD

Une BDD =\> Plusieurs schémas

```
SELECT [...] FROM [data].[Table]
```

Ici, `[data]` est le schéma de `[Table]`

Les schémas servent à regrouper des tables de même nature =\> on peut y affecter des droits (ex: `[ref]` (tables de référence) en RW pour l'admin mais lecture seule pour app, `[data]` (tables de données) en RW pour admin et app mais lecture seule pour le service compta)

In [ ]:
CREATE SCHEMA [ref]

GO

CREATE SCHEMA [data]

GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

## Création de table

In [ ]:
CREATE TABLE [ref].[Genre_GRE] -- [ref] car lecture-seule pour app
(
    [Id] INT IDENTITY(1, 1) PRIMARY KEY,
    [Nom] VARCHAR(25) NOT NULL
)

CREATE TABLE [data].[Film_FLM] -- [data] car lecture-écriture pour app
(
    [Id] BIGINT IDENTITY(1, 1),-- PRIMARY KEY, -- [Id] entier 64bits auto-incrément à partir de 1 par pas de 1, clé primaire
    [Nom] NVARCHAR(80) NOT NULL, -- NVARCHAR gère l'encode, VARCHAR non -> prend 2x plus de place
    [Descriptif] NVARCHAR(MAX) NOT NULL, -- Equivalent à TEXT -> remplacer TEXT par NVARCHAR
    -- [Genre] VARCHAR(20) NOT NULL -> Non, on normalise
    [GRE_Id] INT NOT NULL-- FOREIGN KEY REFERENCES [ref].[Genre_GRE]([Id])
    CONSTRAINT PK_FLM PRIMARY KEY ([Id]) -- On privilégie cette syntaxe -> Contrainte nommée + clé composite
    CONSTRAINT FK_FLM_GRE FOREIGN KEY ([GRE_Id]) REFERENCES [ref].[Genre_GRE]([Id])
)

CREATE TABLE [data].[Acteur_ACT] (
    [Id] UNIQUEIDENTIFIER NOT NULL DEFAULT NEWID() PRIMARY KEY, -- Id 128bits sous forme de chaîne
    [Nom] NVARCHAR(80) NOT NULL,
    [Prenom] NVARCHAR(80) NOT NULL
)

CREATE TABLE [data].[FilmActeur_FAC] (
    [FLM_Id] BIGINT NOT NULL,
    [ACT_Id] UNIQUEIDENTIFIER NOT NULL
    CONSTRAINT PK_FilmActeur PRIMARY KEY ([FLM_Id], [ACT_Id]),
    CONSTRAINT FK_FilmActeur_Acteur FOREIGN KEY ([ACT_Id]) REFERENCES [data].[Acteur_ACT]([Id]),
    CONSTRAINT FK_FilmActeur_Film FOREIGN KEY ([FLM_Id]) REFERENCES [data].[Film_FLM]([Id])
)


Commands completed successfully.

Total execution time: 00:00:00.026

In [ ]:
SELECT NEWID()

(1 row affected)

Total execution time: 00:00:00.009

(No column name)
3b400e68-34e5-4ecd-9548-932460c07c93


## Modification de table

### Exercice

Modéliser des sociétés de production (ex: Lucas Film), les associées aux films (vous aurez besoin de l'instruction [`ALTER TABLE`](https://docs.microsoft.com/fr-fr/sql/t-sql/statements/alter-table-transact-sql?view=sql-server-ver15)). On partira du principe que un film possède une seule société de prod.

In [ ]:
CREATE TABLE [data].[SocieteProd_SPR] (
    [Id] BIGINT IDENTITY(1, 1) PRIMARY KEY,
    [Nom] NVARCHAR(100) NOT NULL
)

GO

ALTER TABLE [data].[Film_FLM]
    ADD [SPR_Id] BIGINT NOT NULL
    CONSTRAINT FK_FML_SPR FOREIGN KEY ([SPR_Id]) REFERENCES [data].[SocieteProd_SPR]([Id])

GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.015

## Opérations CRUD

Create, Read, Update, Delete

  

Create -\> INSERT

Read -\> SELECT

Update -\> UPDATE

Delete -\> DELETE

In [ ]:
INSERT INTO [ref].[Genre_GRE] ([Nom])
VALUES ('Science Fiction'), ('Action'), ('Aventure')

INSERT INTO [data].[SocieteProd_SPR] ([Nom])
VALUES (N'Lucas Film'), (N'Gaumont')

INSERT INTO [data].[Acteur_ACT] ([Nom], [Prenom])
VALUES (N'Ford', N'Harrisson'), (N'Depp', N'Johny')

(3 rows affected)

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.018

In [ ]:
INSERT INTO [data].[Film_FLM] ([Nom], [Descriptif], [GRE_Id], [SPR_Id])
VALUES
    (N'Indiana Jones', N'Descriptif du film', 3, 1),
    (N'Pirates de Caraïbes', N'Descriptif de l''autre film', 2, 2)

(2 rows affected)

Total execution time: 00:00:00.020

In [ ]:
INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 1
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Ford'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 2
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Depp'

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.020

In [ ]:
UPDATE [data].[Film_FLM]
SET [Descriptif] = N'Film sur un archéologue'
WHERE [Id] = 1

UPDATE [data].[Film_FLM]
SET [Descriptif] = N'Film de pirates'
WHERE [Id] = 2


(1 row affected)

(1 row affected)

Total execution time: 00:00:00.016

In [ ]:
DELETE FROM [ref].[Genre_GRE]
WHERE [Id] = 1

(1 row affected)

Total execution time: 00:00:00.012

## Vues

Reppel jointures : https://sql.sh/cours/jointures

In [ ]:
SELECT [FLM].[Id], [FLM].[Nom], [GRE].[Nom] AS [Genre], [SPR].[Nom] AS [Société de prod]
FROM [data].[Film_FLM] AS [FLM]
INNER JOIN [ref].[Genre_GRE] AS [GRE] ON [FLM].[GRE_Id] = [GRE].[Id]
INNER JOIN [data].[SocieteProd_SPR] AS [SPR] ON [FLM].[SPR_Id] = [SPR].[Id]

(2 rows affected)

Total execution time: 00:00:00.018

Id,Nom,Genre,Société de prod
1,Indiana Jones,Aventure,Lucas Film
2,Pirates de Caraïbes,Action,Gaumont


In [ ]:
CREATE OR ALTER VIEW [data].[FicheFilm]
AS
SELECT [FLM].[Id], [FLM].[Nom], [GRE].[Nom] AS [Genre], [SPR].[Nom] AS [Société de prod]
FROM [data].[Film_FLM] AS [FLM]
INNER JOIN [ref].[Genre_GRE] AS [GRE] ON [FLM].[GRE_Id] = [GRE].[Id]
INNER JOIN [data].[SocieteProd_SPR] AS [SPR] ON [FLM].[SPR_Id] = [SPR].[Id]

Commands completed successfully.

Total execution time: 00:00:00.009

In [ ]:
SELECT [Id], [Nom], [Genre], [Société de prod]
FROM [data].[FicheFilm]

(2 rows affected)

Total execution time: 00:00:00.008

Id,Nom,Genre,Société de prod
1,Indiana Jones,Aventure,Lucas Film
2,Pirates de Caraïbes,Action,Gaumont


### Exercice

1\. Insérer de nouveaux acteurs (4), genres, sociétés de prod (3) et de nouveaux films (5) dans vos tables (assignez des acteurs aux films).

2\. Créer une vue qui sort les acteurs avec leurs films associés (Nom de l'acteur, nom du film, genre du film, société de prod)

In [ ]:
INSERT INTO [ref].[Genre_GRE] ([Nom])
VALUES ('Horreur'), ('Science Fiction'), ('Comédie')

INSERT INTO [data].[SocieteProd_SPR] ([Nom])
VALUES (N'Disney'), (N'Netflix'), (N'Pathé')

INSERT INTO [data].[Acteur_ACT] ([Nom], [Prenom])
VALUES (N'De Niro', N'Robert'), (N'Fois', N'Marina'), (N'Cotillard', N'Marion'), (N'Nanty', N'Isabelle')

INSERT INTO [data].[Film_FLM] ([Nom], [Descriptif], [GRE_Id], [SPR_Id])
VALUES
    (N'Les tuches', N'Description tuches', 6, 5),
    (N'Batman', N'Descriptif Batman', 5, 4),
    (N'Rrrrr', N'Descriptif Rrrrr', 6, 1),
    (N'Casino', N'Descriptif Casino', 2, 1),
    (N'Pirates de Caraïbes 2', N'Descriptif de l''autre film', 2, 3)

(3 rows affected)

(3 rows affected)

(4 rows affected)

(5 rows affected)

Total execution time: 00:00:00.031

In [ ]:
INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 3
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Nanty'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 4
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Cotillard'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 5
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Fois'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 6
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'De Niro'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 7
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Depp'

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.032

In [ ]:
CREATE OR ALTER VIEW [data].[ViewActeursFilms] WITH SCHEMABINDING
AS
    SELECT [ACT].[Nom], [FLM].[Nom] AS [Film], [GRE].[Nom] AS [Genre], [SPR].[Nom] AS [Prod]
    FROM [data].[Acteur_ACT] AS [ACT]
    INNER JOIN [data].[FilmActeur_FAC] AS [FAC] ON [FAC].[ACT_Id] = [ACT].[Id]
    INNER JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Id] = [FAC].[FLM_Id]
    INNER JOIN [ref].[Genre_GRE] AS [GRE] ON [GRE].[Id] = [FLM].[GRE_Id]
    INNER JOIN [data].[SocieteProd_SPR] AS [SPR] ON [SPR].[Id] = [FLM].[SPR_Id]

Commands completed successfully.

Total execution time: 00:00:00.053

In [ ]:
SELECT [Nom], [Film], [Genre], [Prod] FROM [data].[ViewActeursFilms]
ORDER BY [Nom]

(7 rows affected)

Total execution time: 00:00:00.033

Nom,Film,Genre,Prod
Cotillard,Batman,Science Fiction,Netflix
De Niro,Casino,Action,Lucas Film
Depp,Pirates de Caraïbes 2,Action,Disney
Depp,Pirates de Caraïbes,Action,Gaumont
Fois,Rrrrr,Comédie,Lucas Film
Ford,Indiana Jones,Aventure,Lucas Film
Nanty,Les tuches,Comédie,Pathé


## Vues Matérialisées

Une vue à un instant T écrite sur disque. =\> Pas à jour en temps réel.

Point fort : Vitesse de lecture instantanée

Point faible : Mise à jour manuelle

## Index

Créer sur des tables ou des vues.

Différents types d'index :

\- Gestion de contraintes (unicité d'une valeur)

\- Faciliter l'accès aux données

\- ...

In [ ]:
CREATE UNIQUE INDEX [IDX_Unique_GRE] ON [ref].[Genre_GRE]([Nom] DESC)

Commands completed successfully.

Total execution time: 00:00:00.013

In [ ]:
--INSERT INTO [ref].[Genre_GRE] ([Nom])
--VALUES ('Horreur') -- Index unique, on a déjà la valeur -> erreur

Commands completed successfully.

Total execution time: 00:00:00.002

In [ ]:
CREATE INDEX [IDX_Acteur_Id_Par_Nom] ON [data].[Acteur_ACT]([Nom], [Id])
CREATE UNIQUE CLUSTERED INDEX [IDX_Vue_Acteur_Film] ON [data].[ViewActeursFilms]([Nom], [Film])

Commands completed successfully.

Total execution time: 00:00:00.023

In [ ]:
SELECT [Id]
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Depp'

(1 row affected)

Total execution time: 00:00:00.011

Id
3ab6b5f1-412f-4bb2-888a-4a3d59606e2b


In [ ]:
SELECT [Film]
FROM [data].[ViewActeursFilms]
WHERE [Nom] = 'Depp'

(2 rows affected)

Total execution time: 00:00:00.013

Film
Pirates de Caraïbes
Pirates de Caraïbes 2


## Fonctions

2 types :

\- Natives

\- UDF (User Defined Functions)

In [ ]:
SELECT TOP 1
    GETDATE() AS [Date],
    DATEADD(DAY, 2, GETDATE()) AS [Après demain],
    DATEDIFF(DAY, DATEADD(MONTH, 2, GETDATE()), GETDATE()) AS [Diff deux mois],
    CONCAT([Prenom], ' ', [Nom]) AS [Nom complet],
    COALESCE(null, 'Test') AS [Coalesce]
FROM [data].[Acteur_ACT]

(1 row affected)

Total execution time: 00:00:00.010

Date,Après demain,Diff deux mois,Nom complet,Coalesce
2021-03-11 18:07:52.703,2021-03-13 18:07:52.703,-61,Johny Depp,Test


In [ ]:
CREATE FUNCTION [data].[fun_GetFilmParNomActeur](@Nom NVARCHAR(80))
RETURNS NVARCHAR(80)
AS
BEGIN
    DECLARE @Film NVARCHAR(80)

    SELECT TOP 1  @Film = [Film]
    FROM [data].[ViewActeursFilms]
    WHERE [Nom] = @Nom

    RETURN @Film
END

Commands completed successfully.

Total execution time: 00:00:00.007

In [ ]:
SELECT [Nom], [data].[fun_GetFilmParNomActeur]([Nom]) AS [Film]
FROM [data].[Acteur_ACT]

(6 rows affected)

Total execution time: 00:00:00.022

Nom,Film
Cotillard,Batman
De Niro,Casino
Depp,Pirates de Caraïbes
Fois,Rrrrr
Ford,Indiana Jones
Nanty,Les tuches


## Exercice
\- Créez une table de référence récompenses avec les valeurs "Oscar du meilleur acteur", "Oscar de la meilleure actrice", "Meilleure bande son"
\- Faire en sorte qu'un film ou un acteur puisse avoir 0 à n récompenses
\- Attribuer des récompenses à deux acteurs et deux films (si possible pas à un acteur du film)
\- Faire une vue qui permet de récupérer les récompenses des acteurs/films
| Nom | Récompense |
| --- | --- |
| Robert De Niro | Oscar du meilleur acteur |
| Les Tuches | Meilleure bande son |
| Batman | NULL |

In [ ]:
CREATE TABLE [ref].[Recompense_RCP] (
    [Id] TINYINT PRIMARY KEY IDENTITY(1, 1),
    [Nom] NVARCHAR(50) NOT NULL UNIQUE
)

CREATE TABLE [data].[FilmsRecompenses_FRC] (
    [FLM_Id] BIGINT NOT NULL,
    [RCP_Id] TINYINT NOT NULL,
    CONSTRAINT PK_FilmsRecompenses PRIMARY KEY ([FLM_Id], [RCP_Id]),
    CONSTRAINT FK_FLM_FRC FOREIGN KEY ([FLM_Id]) REFERENCES [data].[Film_FLM]([Id]),
    CONSTRAINT FK_RCP_FRC FOREIGN KEY ([RCP_Id]) REFERENCES [ref].[Recompense_RCP]([Id])
)

CREATE TABLE [data].[ActeursRecompenses_ARC] (
    [ACT_Id] UNIQUEIDENTIFIER NOT NULL,
    [FLM_Id] BIGINT NOT NULL,
    [RCP_Id] TINYINT NOT NULL,
    CONSTRAINT PK_ActeursRecompenses PRIMARY KEY ([ACT_Id], [FLM_Id], [RCP_Id]),
    CONSTRAINT FK_ACT_ARC FOREIGN KEY ([ACT_Id]) REFERENCES [data].[Acteur_ACT]([Id]),
    CONSTRAINT FK_FLM_ARC FOREIGN KEY ([FLM_Id]) REFERENCES [data].[Film_FLM]([Id]),
    CONSTRAINT FK_RCP_ARC FOREIGN KEY ([RCP_Id]) REFERENCES [ref].[Recompense_RCP]([Id])
)


Commands completed successfully.

Total execution time: 00:00:00.023

In [ ]:
INSERT INTO [ref].[Recompense_RCP] ([Nom])
VALUES (N'Oscar du meilleur acteur'), (N'Oscar de la meilleure actrice'), (N'Oscar de la meilleure BO')

INSERT INTO [data].[FilmsRecompenses_FRC] ([FLM_Id], [RCP_Id])
VALUES (1, 3)

INSERT INTO [data].[ActeursRecompenses_ARC] ([ACT_Id], [FLM_Id], [RCP_Id])
SELECT TOP 1 [ACT].[Id], [FLM].[Id], 1
FROM [data].[ViewActeursFilms] AS [VUE]
INNER JOIN [data].[Acteur_ACT] AS [ACT] ON [ACT].[Nom] = [VUE].[Nom]
INNER JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Nom] = [VUE].[Film]
WHERE [ACT].[NOM] = 'Ford'

INSERT INTO [data].[ActeursRecompenses_ARC] ([ACT_Id], [FLM_Id], [RCP_Id])
SELECT TOP 1 [ACT].[Id], [FLM].[Id], 2
FROM [data].[ViewActeursFilms] AS [VUE]
INNER JOIN [data].[Acteur_ACT] AS [ACT] ON [ACT].[Nom] = [VUE].[Nom]
INNER JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Nom] = [VUE].[Film]
WHERE [ACT].[NOM] = 'Nanty'

(3 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.046

In [ ]:
CREATE OR ALTER VIEW [data].[VueRecompenses]
AS
    SELECT
        [RCP].[Nom] AS [Récompense], 
        COALESCE([FLM].[Nom], CONCAT([ACT].[Nom], ' pour ', [FLM2].[Nom])) AS [Nom]
    FROM [ref].[Recompense_RCP] AS [RCP]
    LEFT JOIN [data].[FilmsRecompenses_FRC] AS [FRC] ON [FRC].[RCP_Id] = [RCP].[Id]
    LEFT JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Id] = [FRC].[FLM_Id]

    LEFT JOIN [data].[ActeursRecompenses_ARC] AS [ARC] ON [ARC].[RCP_Id] = [RCP].[Id]
    LEFT JOIN [data].[Acteur_ACT] AS [ACT] ON [ACT].[Id] = [ARC].[ACT_Id]
    LEFT JOIN [data].[Film_FLM] AS [FLM2] ON [FLM2].[Id] = [ARC].[FLM_Id]

Commands completed successfully.

Total execution time: 00:00:00.008

In [ ]:
SELECT * FROM [data].[VueRecompenses]

(3 rows affected)

Total execution time: 00:00:00.043

Récompense,Nom
Oscar de la meilleure actrice,Nanty pour Les tuches
Oscar de la meilleure BO,Indiana Jones
Oscar du meilleur acteur,Ford pour Indiana Jones


## Les triggers

Un trigger (déclencheur) =\> un code éxécuté sur un événement donné

Curseur =\> Une sorte de référence sur un espace mémoire qui comporte un résultat de requête et qui nous permet de le parcourir enregistrement par enregistrement

In [ ]:
ALTER TABLE [data].[Film_FLM]
    ADD [LastEdit] DATETIME NOT NULL DEFAULT GETDATE()

Commands completed successfully.

Total execution time: 00:00:00.007

In [ ]:
SELECT * FROM [data].[Film_FLM]

(7 rows affected)

Total execution time: 00:00:00.011

Id,Nom,Descriptif,GRE_Id,SPR_Id,LastEdit
1,Indiana Jones,Film sur un archéologue,3,1,2021-03-11 18:07:56.420
2,Pirates de Caraïbes,Film de pirates,2,2,2021-03-11 18:07:56.420
3,Les tuches,Description tuches,6,5,2021-03-11 18:07:56.420
4,Batman,Descriptif Batman,5,4,2021-03-11 18:07:56.420
5,Rrrrr,Descriptif Rrrrr,6,1,2021-03-11 18:07:56.420
6,Casino,Descriptif Casino,2,1,2021-03-11 18:07:56.420
7,Pirates de Caraïbes 2,Descriptif de l'autre film,2,3,2021-03-11 18:07:56.420


In [ ]:
CREATE OR ALTER TRIGGER [data].[trig_UpdateFLMLastUpdate]
    ON [data].[Film_FLM]
    AFTER UPDATE
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @FlmId BIGINT
    DECLARE @NbItems INT

    SELECT @NbItems = COUNT(*) FROM INSERTED

    IF @NbItems > 1
    BEGIN
        DECLARE curs CURSOR FOR SELECT [Id] FROM INSERTED

        OPEN curs

        FETCH NEXT FROM curs INTO @FlmId
        WHILE (@@FETCH_STATUS = 0)
        BEGIN
            UPDATE [data].[Film_FLM]
            SET [LastEdit] = GETDATE()
            WHERE [Id] = @FlmId

            FETCH NEXT FROM curs INTO @FlmId
        END

        CLOSE curs
        DEALLOCATE curs
    END
    ELSE
    BEGIN
        SELECT TOP 1 @FlmId = [Id] FROM INSERTED

        UPDATE [data].[Film_FLM]
        SET [LastEdit] = GETDATE()
        WHERE [Id] = @FlmId
    END
END

Commands completed successfully.

Total execution time: 00:00:00.009

Apparté : Itérateur

  

Itérateur =\> structure qui va vous permettre de faire des boucles "à la main"

1. Déclarer/Récupérer l'itérateur
2. Demander le next / Demander si le next existe
3. Si le next est définit, on le récupère
4. Faire votre traitement
5. Retourner en étape 2

  

## Les transactions

On va créer un contexte de travail pour le SGBD =\> on écrit pas directement les valeurs sur le disque, on peut calculer des valeurs (Ids, ...), mais tant que la transaction n'est pas validée, rien n'est persisté

In [ ]:
BEGIN TRANSACTION MA_TRANSACTION
BEGIN TRY
    INSERT INTO [data].[ActeursRecompenses_ARC] ([ACT_Id], [FLM_Id], [RCP_Id])
    SELECT TOP 1 [ACT].[Id], [FLM].[Id], 2
    FROM [data].[ViewActeursFilms] AS [VUE]
    INNER JOIN [data].[Acteur_ACT] AS [ACT] ON [ACT].[Nom] = [VUE].[Nom]
    INNER JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Nom] = [VUE].[Film]
    WHERE [ACT].[NOM] = 'Fois'

    INSERT INTO [data].[ActeursRecompenses_ARC] ([ACT_Id], [FLM_Id], [RCP_Id])
    SELECT TOP 1 [ACT].[Id], [FLM].[Id], 2
    FROM [data].[ViewActeursFilms] AS [VUE]
    INNER JOIN [data].[Acteur_ACT] AS [ACT] ON [ACT].[Nom] = [VUE].[Nom]
    INNER JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Nom] = [VUE].[Film]
    WHERE [ACT].[NOM] = 'Nanty'

    COMMIT TRANSACTION MA_TRANSACTION
END TRY
BEGIN CATCH
    ROLLBACK TRANSACTION MA_TRANSACTION

    SELECT ERROR_NUMBER(), ERROR_MESSAGE()
END CATCH

SELECT * FROM [data].[VueRecompenses]

(1 row affected)

(0 rows affected)

(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.059

(No column name),(No column name)
2627,"Violation of PRIMARY KEY constraint 'PK_ActeursRecompenses'. Cannot insert duplicate key in object 'data.ActeursRecompenses_ARC'. The duplicate key value is (b24198de-f8e4-4afb-8d7e-f2404a03dd14, 3, 2)."


Récompense,Nom
Oscar de la meilleure actrice,Nanty pour Les tuches
Oscar de la meilleure BO,Indiana Jones
Oscar du meilleur acteur,Ford pour Indiana Jones


In [ ]:
BEGIN TRANSACTION MA_TRANSACTION
BEGIN TRY
    INSERT INTO [data].[ActeursRecompenses_ARC] ([ACT_Id], [FLM_Id], [RCP_Id])
    SELECT TOP 1 [ACT].[Id], [FLM].[Id], 2
    FROM [data].[ViewActeursFilms] AS [VUE]
    INNER JOIN [data].[Acteur_ACT] AS [ACT] ON [ACT].[Nom] = [VUE].[Nom]
    INNER JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Nom] = [VUE].[Film]
    WHERE [ACT].[NOM] = 'Fois'

    INSERT INTO [data].[ActeursRecompenses_ARC] ([ACT_Id], [FLM_Id], [RCP_Id])
    SELECT TOP 1 [ACT].[Id], [FLM].[Id], 2
    FROM [data].[ViewActeursFilms] AS [VUE]
    INNER JOIN [data].[Acteur_ACT] AS [ACT] ON [ACT].[Nom] = [VUE].[Nom]
    INNER JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Nom] = [VUE].[Film]
    WHERE [ACT].[NOM] = 'Cotillard'

    COMMIT TRANSACTION MA_TRANSACTION
END TRY
BEGIN CATCH
    ROLLBACK TRANSACTION MA_TRANSACTION

    SELECT ERROR_NUMBER(), ERROR_MESSAGE()
END CATCH

SELECT * FROM [data].[VueRecompenses]

(1 row affected)

(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.031

Récompense,Nom
Oscar de la meilleure actrice,Cotillard pour Batman
Oscar de la meilleure actrice,Fois pour Rrrrr
Oscar de la meilleure actrice,Nanty pour Les tuches
Oscar de la meilleure BO,Indiana Jones
Oscar du meilleur acteur,Ford pour Indiana Jones


## Procédures stockées

Elles vont nous permettre de stocker des suites d'opération dont le code va être optimisé pour le SGBD =\> grande vitesse d'éxécution

In [ ]:
CREATE OR ALTER PROCEDURE [ref].[prc_InsertionActeur]
    @Nom NVARCHAR(80),
    @Prenom NVARCHAR(80)
AS
BEGIN
    SET NOCOUNT ON;

    BEGIN TRANSACTION InsertAct
    BEGIN TRY
        INSERT INTO [data].[Acteur_ACT] ([Nom], [Prenom])
        VALUES (@Nom, @Prenom)

        COMMIT TRANSACTION InsertAct

        SELECT 0 AS [Number], N'' AS [Message]
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION InsertAct

        SELECT ERROR_NUMBER() AS [Number], ERROR_MESSAGE() AS [Message]
    END CATCH
END;

Commands completed successfully.

Total execution time: 00:00:00.017

In [ ]:
CREATE OR ALTER PROCEDURE [ref].[prc_InsertArc]
    @ActId UNIQUEIDENTIFIER,
    @FlmId BIGINT,
    @RcpId TINYINT
AS
BEGIN
    SET NOCOUNT ON;

    BEGIN TRANSACTION InsertArc
    BEGIN TRY
        INSERT INTO [data].[ActeursRecompenses_ARC] ([ACT_Id], [FLM_Id], [RCP_Id])
        VALUES (@ActId, @FlmId, @RcpId)

        COMMIT TRANSACTION InsertArc

        SELECT 0 AS [Number], N'' AS [Message]
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION InsertArc

        SELECT ERROR_NUMBER() AS [Number], ERROR_MESSAGE() AS [Message]
    END CATCH
END

Commands completed successfully.

Total execution time: 00:00:00.049

In [ ]:
DECLARE @Id UNIQUEIDENTIFIER
SELECT @Id = NEWID()

EXEC [ref].[prc_InsertArc] @ActId = @Id, @FlmId = 12, @RcpId = 12

Commands completed successfully.

Total execution time: 00:00:00.009

Number,Message
547,"The INSERT statement conflicted with the FOREIGN KEY constraint ""FK_ACT_ARC"". The conflict occurred in database ""ImdbLike"", table ""data.Acteur_ACT"", column 'Id'."
